In [1]:
import glob
import matplotlib as mlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
from numpy import loadtxt
import re
import csv
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.patches as mpatches

In [7]:
# d0 for lectrons and muons separately\n
def impact_d0(folder):
    #reading pythia output
    files = sorted(glob.glob( folder+'/*.hepmc*'))
    print(files)

    
    
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        d0_l=[]
        d0_e=[]
        d0_mu=[]
        pT_e=[]
        pT_mu=[]
        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc))   
        counte=0
        countmu=0
        countV=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='V'):
                countV+=1
                #print('V=',row)
                ip=rownum+1
                #finding electron/muon related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        
                        if (hepmc[ip+i][2]=='13'):
                            countmu+=1
                            #print("mu=",hepmc[ip+i])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating |d0|
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            if (VT!=0): 
                                PT=np.sqrt(P[0]**2+P[1]**2)

                                cTheta=(V[0]*P[0]+V[1]*P[1])/(VT*PT)
                                
                                pT_mu.append(PT)
                                d0_mu.append(PT*np.sqrt(1-cTheta**2))
                                
                        if (hepmc[ip+i][2]=='11'):
                            counte+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating |d0|
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            if (VT!=0): 
                                PT=np.sqrt(P[0]**2+P[1]**2)

                                cTheta=(V[0]*P[0]+V[1]*P[1])/(VT*PT)
                                
                                pT_e.append(PT)
                                d0_e.append(PT*np.sqrt(1-cTheta**2))
                            


                        i+=1
        print("counte={}".format(counte)+" countmu={}".format(countmu)+" countV={}".format(countV)+"\n")
        #print(d0_l)
    
        
        
        #Saving d0 to file
        with open("hepmc_output/d0_e.dat", 'a+') as foute:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(foute, d0_e, fmt='%7f')   
                    
        with open("hepmc_output/d0_mu.dat", 'a+') as foutmu:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(foutmu, d0_mu, fmt='%7f')
                    
        #Saving pT to file
        with open("hepmc_output/pT_e.dat", 'a+') as foute:
                        np.savetxt(foute, pT_e, fmt='%7f')   
                    
        with open("hepmc_output/pT_mu.dat", 'a+') as foutmu:
                        np.savetxt(foutmu, pT_mu, fmt='%7f')

In [9]:
#choosing var1 & var2 with respect to cut on var1

def chooseWithCut(file1,file2,variable1,variable2,output_folder,cut_file1):
    var1 = np.array(genfromtxt(file1))
    var2 = np.array(genfromtxt(file2))
    
    print(var2[var1>cut_file1])
    
    var1New=var1[var1>cut_file1]
    var2New=var2[var1>cut_file1]
    
    with open(output_folder+"_cut"+variable1, 'w') as fout1:
        np.savetxt(fout1, var1New, fmt='%7f')
        
    with open(output_folder+"_cut"+variable2, 'w') as fout2:
        np.savetxt(fout2, var2New, fmt='%7f')

In [12]:
impact_d0('hepmc')

['hepmc/test.hepmc']
hepmc/test.hepmc
5353
mu= ['P', '159', '13', '4.0913961294246368e+00', '-3.5952608201326917e+00', '-7.6628374331835118e+00', '9.4018968413640120e+00', '1.0566000000000000e-01', '1', '0', '0', '0', '0']
mu= ['P', '1233', '13', '1.1698544106988020e+01', '3.0832388396791210e+00', '-1.1663742373246626e+00', '1.2154583039444203e+01', '1.0566000000000000e-01', '1', '0', '0', '0', '0']
mu= ['P', '203', '13', '-2.0015523362364480e+01', '-5.1429146193120179e+00', '7.2901768302773124e+00', '2.1914118474244038e+01', '1.0566000000000000e-01', '1', '0', '0', '0', '0']
counte=2 countmu=3 countV=2024



In [10]:
impact_d0('hepmc_large')

['hepmc_large/pythia-33000events_split_1.hepmcaa', 'hepmc_large/pythia-33000events_split_1.hepmcab', 'hepmc_large/pythia-33000events_split_1.hepmcac', 'hepmc_large/pythia-33000events_split_1.hepmcad', 'hepmc_large/pythia-33000events_split_1.hepmcae', 'hepmc_large/pythia-33000events_split_1.hepmcaf', 'hepmc_large/pythia-33000events_split_1.hepmcag', 'hepmc_large/pythia-33000events_split_1.hepmcah', 'hepmc_large/pythia-33000events_split_1.hepmcai', 'hepmc_large/pythia-33000events_split_1.hepmcaj', 'hepmc_large/pythia-33000events_split_1.hepmcak', 'hepmc_large/pythia-33000events_split_1.hepmcal', 'hepmc_large/pythia-33000events_split_1.hepmcam', 'hepmc_large/pythia-33000events_split_1.hepmcan', 'hepmc_large/pythia-33000events_split_1.hepmcao', 'hepmc_large/pythia-33000events_split_1.hepmcap', 'hepmc_large/pythia-33000events_split_1.hepmcaq', 'hepmc_large/pythia-33000events_split_1.hepmcar', 'hepmc_large/pythia-33000events_split_1.hepmcas', 'hepmc_large/pythia-33000events_split_1.hepmcat',